# Calculate historical Vapor Pressure Deficit (VPD) grids.

Do this for gridded ERA-Interim data using the estimated RH% field. VPD is a simple forula. 

\begin{align}
VPD & = e_{sat} - e_{actual}
\end{align}

where:

\begin{align}
e_{actual}=\frac{e_{sat}*RH)}{100}
\end{align}

where the saturation water vopor pressure ($e_{sat}$) is expressed with the Teten's formula:
Teton's formula, eq. 7.5 
https://www.ecmwf.int/sites/default/files/elibrary/2015/9211-part-iv-physical-processes.pdf

\begin{align}
e_{sat}(T) & = a_{1}exp(a_{3}\frac{T - T_{0}}{T-a_{4}})\\
\\
a_{1} & = 611.21Pa \\
a_{3} & = 17.502 \\
a_{4} & = 32.19 \\
T_{0} & = 273.16
\end{align}

In [1]:
import numpy as np
from netCDF4 import Dataset
import os

def calculate_es(T):
    """Calculate saturation vapor pressure using Tetons formula"""
    a1 = 611.21 # Pa
    a3 = 17.502  
    a4 = 32.19
    T0 = 273.16 # K
    e_sat = a1 * np.exp( a3 * (T-T0)/(T-a4) )
    
    return e_sat

In [23]:
def get_nc(var, year, dataDir): 
    loadFile = os.path.join(dataDir, var + "_" + str(year) + ".nc")
    nc = Dataset(loadFile)
    vals = nc.variables[var][:]
    t = nc.variables["time"][:]
    lon = nc.variables["longitude"][:]
    lat = nc.variables["latitude"][:]
    nc.close()
    return vals, t, lon, lat

In [22]:
def write_VPD_nc(VALS, t, x, y, year, dataDir):
    
    outputFile = os.path.join(dataDir, "VPD_" + str(year) + ".nc")

    ncFile = Dataset(outputFile, 'w', format='NETCDF4')
    ncFile.description = 'VPD (saturation pressure relative to water, Tetons eq.)'
    ncFile.location = 'Global'
    ncFile.createDimension('time',  len(t) )
    ncFile.createDimension('latitude', len(y) )
    ncFile.createDimension('longitude', len(x) )

    VAR_ = ncFile.createVariable("VPD", 'f4',('time', 'latitude','longitude'))
    VAR_.long_name = "Vapor Pressure Deficit"
    VAR_.units = "Pa"

    # Create time variable
    time_ = ncFile.createVariable('time', 'i4', ('time',))
    time_.units = "hours since 1900-01-01 00:00:0.0"
    time_.calendar = "gregorian"
    
    # create lat variable
    latitude_ = ncFile.createVariable('latitude', 'f4', ('latitude',))
    latitude_.units = 'degrees_north'

    # create longitude variable
    longitude_ = ncFile.createVariable('longitude', 'f4', ('longitude',))
    longitude_.units = 'degrees_east'

    # Write the actual data to these dimensions
    VAR_[:]       = VALS
    time_[:]      = t
    latitude_[:]  = y
    longitude_[:] = x

    ncFile.close()

### Use above functions to create VPD file. 

In [25]:
dataDir = os.path.join("..","..", "metSpreadData", "ERA-Interim")
year = "1990-2015"
    
# Get temperature to calculate es
t2m,t,x,y = get_nc("t2m", year, dataDir)
    
# Get RH% to calculate e_actual
RH,t,x,y = get_nc("RH", year, dataDir)

# get saturation and actual vapor pressure 
e_sat = calculate_es(t2m)
e_actual = e_sat * (RH / 100.)

# Calculate VPD using what is known as subtraction
VPD = e_sat - e_actual

write_VPD_nc(VPD, t, x, y, year, dataDir)

Use cdo remapbil,COMMON_GRID.txt to create the needed file for comparing to CMIP5. Move that file to the metSpread directory. 

### Do the same for CMIP5 output. 

In [23]:
import model_dictionary as md

In [24]:
def get_CMIP_nc(var, loadFile): 
    
    nc = Dataset(loadFile)
    vals = nc.variables[var][:]
    t = nc.variables["time"][:]
    lon = nc.variables["lon"][:]
    lat = nc.variables["lat"][:]
    nc.close()
    return vals, t, lon, lat

In [25]:
def write_VPD_CMIP5_nc(VALS, model, rcp, t, x, y, dataDir):
    
    outputFile = os.path.join(dataDir, "VPD_Amon_"+model+"_"+rcp+"_r1i1p1_200601-210012.nc")

    ncFile = Dataset(outputFile, 'w', format='NETCDF4')
    ncFile.description = 'VPD (saturation pressure relative to water, Tetons eq.)'
    ncFile.location = 'Global'
    ncFile.createDimension('time',  len(t) )
    ncFile.createDimension('lat', len(y) )
    ncFile.createDimension('lon', len(x) )

    VAR_ = ncFile.createVariable("VPD", 'f4',('time', 'lat','lon'))
    VAR_.long_name = "Vapor Pressure Deficit"
    VAR_.units = "Pa"

    # Create time variable
    time_ = ncFile.createVariable('time', 'i4', ('time',))
    time_.units = "days since 2006-1-1"
    #time_.calendar = "gregorian"
    
    # create lat variable
    latitude_ = ncFile.createVariable('lat', 'f4', ('lat',))
    latitude_.units = 'degrees_north'
    latitude_.axis = "Y"

    # create longitude variable
    longitude_ = ncFile.createVariable('lon', 'f4', ('lon',))
    longitude_.units = 'degrees_east'
    longitude_.axis = "X"

    # Write the actual data to these dimensions
    VAR_[:]       = VALS
    time_[:]      = t
    latitude_[:]  = y
    longitude_[:] = x

    ncFile.close()

In [27]:
dataDir = os.path.join("..", "Data", "CMIP5", "r1i1p1_rcp45_rcp85_merged_t_COMMON_GRID")
print dataDir

rcps = ['rcp45', 'rcp85']
for rcp in rcps : 
    
    print "Working on: " + rcp
    
    for model in md.model_name.keys() :
    
        print model
                
        # see if Temperature and RH file exist
        T_file = os.path.join(dataDir, "tas_Amon_"+model+"_"+rcp+"_r1i1p1_200601-210012.nc")
        RH_file = os.path.join(dataDir, "hurs_Amon_"+model+"_"+rcp+"_r1i1p1_200601-210012.nc")

        # If these files exist (they don't for all models and rcps), continue
        if os.path.isfile(T_file) & os.path.isfile(RH_file) : 
            
            # Get the values to calculate e
            t2m,t,x,y = get_CMIP_nc('tas', T_file)
            RH,t,x,y = get_CMIP_nc('hurs', RH_file)

            # get saturation and actual vapor pressure 
            e_sat = calculate_es(t2m)
            e_actual = e_sat * (RH / 100.)

            # Calculate VPD using what is known as subtraction
            VPD = e_sat - e_actual

            write_VPD_CMIP5_nc(VPD, model, rcp, t, x, y,  dataDir)

../Data/CMIP5/r1i1p1_rcp45_rcp85_merged_t_COMMON_GRID
Working on: rcp45
ACCESS1-0
ACCESS1-3
IPSL-CM5B-LR
CMCC-CMS
MIROC-ESM
MIROC5
CanESM2
MPI-ESM-MR
CSIRO-Mk3-6-0
inmcm4
GISS-E2-R-CC
CCSM4
GFDL-ESM2G
CMCC-CM
GFDL-ESM2M
NorESM1-M
IPSL-CM5A-MR
IPSL-CM5A-LR
GFDL-CM3
CNRM-CM5
GISS-E2-H
MIROC-ESM-CHEM
NorESM1-ME
HadGEM2-ES
GISS-E2-R
HadGEM2-CC
GISS-E2-H-CC
CMCC-CESM
MRI-CGCM3
MPI-ESM-LR
MRI-ESM1
Working on: rcp85
ACCESS1-0
ACCESS1-3
IPSL-CM5B-LR
CMCC-CMS
MIROC-ESM
MIROC5
CanESM2
MPI-ESM-MR
CSIRO-Mk3-6-0
inmcm4
GISS-E2-R-CC
CCSM4
GFDL-ESM2G
CMCC-CM
GFDL-ESM2M
NorESM1-M
IPSL-CM5A-MR
IPSL-CM5A-LR
GFDL-CM3
CNRM-CM5
GISS-E2-H
MIROC-ESM-CHEM
NorESM1-ME
HadGEM2-ES
GISS-E2-R
HadGEM2-CC
GISS-E2-H-CC
CMCC-CESM
MRI-CGCM3
MPI-ESM-LR
MRI-ESM1
